<a href="https://colab.research.google.com/github/PreyPython123/Master-V24-Semiveiledet-Regresjon/blob/Mw-Pradeep/Bioco_Mw_Superveiledet_Regresjonsmetoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importering, lasting og formattering

Lasting av nødvendige bibliotek og pakker

In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Importering av nødvendig bibliotek og pakker

In [3]:
import pandas as pd
import numpy as np
import copy
import math
from statistics import mean, stdev

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

In [4]:
# Modeller
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [5]:
# Hyperoptimalisering
import optuna
from optuna.visualization import plot_optimization_history

# Utelukker utskriving av logg for hyperoptimalisering
# optuna.logging.set_verbosity(optuna.logging.WARNING)

# Egendefinerte moduler

In [6]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/oppdeling_trening_testsett.py /content/oppdeling_trening_testsett.py

/content


In [7]:
from oppdeling_trening_testsett import trening_testsett_oppdeling_enzym
from oppdeling_trening_testsett import trening_testsett_oppdeling_dag
from oppdeling_trening_testsett import trening_validering_oppdeling_dag


from oppdeling_trening_testsett import enzymfordeling

In [8]:
!pwd
! cp /content/drive/MyDrive/MasterV24/GitHub/moduler_optimalisering_evaluering_veiledet.py /content/moduler_optimalisering_evaluering_veiledet.py

/content


In [9]:
from moduler_optimalisering_evaluering_veiledet import trening_rfr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_knr_enzym
from moduler_optimalisering_evaluering_veiledet import trening_svr_enzym

from moduler_optimalisering_evaluering_veiledet import trening_rfr_dag
from moduler_optimalisering_evaluering_veiledet import trening_knr_dag
from moduler_optimalisering_evaluering_veiledet import trening_svr_dag

from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_1
from moduler_optimalisering_evaluering_veiledet import veiledet_modell_evaluering_2

# Importering av relevant data

In [13]:
# Velger første kolonne med dato og tid som index
data = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data.csv',
                            header=0,
                            sep=',',
                            index_col=0)

# Normal produksjon
data_normal = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_normal.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Design produksjon
data_design = pd.read_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_data_design.csv',
                                   header=0,
                                   sep=',',
                                   index_col=0)

# Importerer rådata
rå_data = pd.read_csv('/content/drive/MyDrive/MasterV24/BiocoData.csv',
                      header=0,
                      sep=';',
                      index_col=0)

# Formatterer index til riktig format og datatype
data.index = pd.to_datetime(data.index,
                            format='%Y-%m-%d %H:%M:%S')

data_normal.index = pd.to_datetime(data_normal.index,
                                   format='%Y-%m-%d %H:%M:%S')

data_design.index = pd.to_datetime(data_design.index,
                                   format='%Y-%m-%d %H:%M:%S')

rå_data.index = pd.to_datetime(rå_data.index,
                               format='%d-%m-%Y %H:%M:%S.%f')

Tilfeldighetsfrø

In [14]:
#Tilfeldighetsfrø
random_seed = 123

# Beskrivelse av datasett

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 28976 entries, 2022-10-31 17:37:00 to 2023-06-14 01:02:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  28976 non-null  int64  
 1   EnzymeType_A2  28976 non-null  int64  
 2   EnzymeType_B   28976 non-null  int64  
 3   EnzymeType_C   28976 non-null  int64  
 4   EnzymeType_D   28976 non-null  int64  
 5   EnzymeType_E   28976 non-null  int64  
 6   RawMatPercent  28976 non-null  float64
 7   NIRfat         28976 non-null  float64
 8   NIRash         28976 non-null  float64
 9   NIRwater       28976 non-null  float64
 10  TT08           28976 non-null  float64
 11  TT20           28976 non-null  float64
 12  TT12           28976 non-null  float64
 13  Mw             455 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 3.3 MB


In [16]:
data_design.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10466 entries, 2022-10-31 17:37:00 to 2023-06-13 23:59:00
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   EnzymeType_A1  10466 non-null  int64  
 1   EnzymeType_A2  10466 non-null  int64  
 2   EnzymeType_B   10466 non-null  int64  
 3   EnzymeType_C   10466 non-null  int64  
 4   EnzymeType_D   10466 non-null  int64  
 5   EnzymeType_E   10466 non-null  int64  
 6   RawMatPercent  10466 non-null  float64
 7   NIRfat         10466 non-null  float64
 8   NIRash         10466 non-null  float64
 9   NIRwater       10466 non-null  float64
 10  TT08           10466 non-null  float64
 11  TT20           10466 non-null  float64
 12  TT12           10466 non-null  float64
 13  Mw             332 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 1.2 MB


# Lagring av resultater

In [18]:
# Kolonner til dataframe for lagring av resultater fra opmtimalisering, trening
# validering og testing

dict_res_1_hel_test = {"modell": [],
                       "test_andel": [],
                       "trening_andel": [],
                       "trening_markert_antall": [],
                       "tid_trening": [],
                       "parametere": [],
                       "RMSE_test": [],
                       "R2_test":[],
                       "MAE_test":[],
                       "MAPE_test":[],
                       "n_runder": [],
                       "n_splitt": [],
                       "tid_optimalisering": []}

dict_res_1_design_test = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_hel_test = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_design_test = copy.deepcopy(dict_res_1_hel_test)

# Uten NIR
dict_res_1_hel_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_1_design_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_hel_test_NIR = copy.deepcopy(dict_res_1_hel_test)
dict_res_2_design_test_NIR = copy.deepcopy(dict_res_1_hel_test)


dict_res_1_hel_trening = {"modell": [],
                          "test_andel": [],
                          "trening_andel": [],
                          "parametere": [],
                          "n_splitt": [],
                          "RMSE_val_snitt": [],
                          "RMSE_val_std": [],
                          "R2_val_snitt":[],
                          "R2_val_std":[],
                          "MAE_val_snitt":[],
                          "MAE_val_std":[],
                          "MAPE_val_snitt":[],
                          "MAPE_val_std":[],
                          "RMSE_trening_snitt": [],
                          "RMSE_trening_std": [],
                          "R2_trening_snitt":[],
                          "R2_trening_std":[],
                          "MAE_trening_snitt":[],
                          "MAE_trening_std":[],
                          "MAPE_trening_snitt":[],
                          "MAPE_trening_std":[],
                          }

dict_res_1_design_trening = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_hel_trening = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_design_trening = copy.deepcopy(dict_res_1_hel_trening)

# Uten NIR
dict_res_1_hel_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_1_design_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_hel_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)
dict_res_2_design_trening_NIR = copy.deepcopy(dict_res_1_hel_trening)

# NIR målinger inkludert

## Hele datasett

## Alternativ 1: Hele treningssett

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, \
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                             trening_dict_res=dict_res_1_hel_trening,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50


# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_1(test_dict_res=dict_res_1_hel_test,
                              trening_dict_res=dict_res_1_hel_trening,
                              treningssett_markert=treningssett_markert,
                              enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                              treningssett_umarkert=treningssett_umarkert,
                              enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                              treningssett=treningssett,
                              testsett=testsett,
                              splitt=splitt,
                              gjentagelser=gjentagelser,
                              n_runder=n_runder,
                              test_andel=test_andel,
                              trenings_andel=trenings_andel,
                              random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        31  34.07
E         15  16.48
B         14  15.38
C         14  15.38
D         13  14.29
A1         4   4.40


Lagring av resultater

In [ ]:
mw_resultater_1_hel_trening = pd.DataFrame(dict_res_1_hel_trening)
mw_resultater_1_hel_test = pd.DataFrame(dict_res_1_hel_test)

mw_resultater_1_hel_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_trening.csv')
mw_resultater_1_hel_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_test.csv')

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer trenings- og testandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Parametervalg for kryssvalidering
splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                            trening_dict_res=dict_res_2_hel_trening,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=trening_umarkert,
                            treningssett=trening,
                            testsett=testsett,
                            splitt=splitt,
                            n_runder=n_runder,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_2(test_dict_res=dict_res_2_hel_test,
                                trening_dict_res=dict_res_2_hel_trening,
                                rå_data=rå_data,
                                treningssett_markert=treningssett_markert,
                                treningssett_umarkert=treningssett_umarkert,
                                treningssett=treningssett,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
A2        25  27.78
B         16  17.78
C         15  16.67
E         15  16.67
D         13  14.44
A1         6   6.67


Lagring av resultater

In [ ]:
mw_resultater_2_hel_trening = pd.DataFrame(dict_res_2_hel_trening)
mw_resultater_2_hel_test = pd.DataFrame(dict_res_2_hel_test)

mw_resultater_2_hel_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_trening.csv')
mw_resultater_2_hel_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_test.csv')

## Designproduksjon

## Alternativ 1: Hele treningssettet

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10


# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                             trening_dict_res=dict_res_1_design_trening,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 1: Forskjellige treningsandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
treningsandeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in treningsandeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, enzymtyper_treningssett_markert,\
      _, _, treningssett_umarkert, _, treningssett\
      = trening_testsett_oppdeling_enzym(rå_data,
                                         trening,
                                         test_andel=ubenyttet_andel,
                                         random_seed=random_seed)

  else:
    treningssett_markert = trening_markert
    enzymtyper_treningssett_markert = enzymtyper_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  # Parameter til kryssvalidering
  gjentagelser = 10

  # Finner antall av enzymtype med minst antall
  enzym_markert_info = enzymfordeling(rå_data, treningssett_markert)
  enzymtype_min = enzym_markert_info['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_1(test_dict_res=dict_res_1_design_test,
                              trening_dict_res=dict_res_1_design_trening,
                              treningssett_markert=treningssett_markert,
                              enzymtyper_treningssett_markert=enzymtyper_treningssett_markert,
                              treningssett_umarkert=treningssett_umarkert,
                              enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                              treningssett=treningssett,
                              testsett=testsett,
                              splitt=splitt,
                              gjentagelser=gjentagelser,
                              n_runder=n_runder,
                              test_andel=test_andel,
                              trenings_andel=trenings_andel,
                              random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.39
B         13  19.40
C         13  19.40
D         12  17.91
A2        10  14.93
A1         4   5.97


Lagring av resultater

In [ ]:
mw_resultater_1_design_trening = pd.DataFrame(dict_res_1_design_trening)
mw_resultater_1_design_test = pd.DataFrame(dict_res_1_design_test)

mw_resultater_1_design_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_trening.csv')
mw_resultater_1_design_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_test.csv')

## Alternativ 2: Hele treningssettet

In [ ]:
# Definerer trenings- ogtestandel
test_andel = 0.2
trenings_andel = 1.0

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)


#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                                trening_dict_res=dict_res_2_design_trening,
                                rå_data=rå_data,
                                treningssett_markert=trening_markert,
                                treningssett_umarkert=trening_umarkert,
                                treningssett=trening,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Velger beste modell

## Alternativ 2: Forskjellige treningssandeler

In [ ]:
# Antall runder til optimalisering
n_runder = 50

# Treningsandeler
trenings_andeler = [0.2, 0.4, 0.6, 0.8]

# Går gjennom andeler av satt treningssett
for trenings_andel in trenings_andeler:

  if trenings_andel < 1.0:

      # Utelukker resterende andel fra trening av modell
      ubenyttet_andel = (1.0-trenings_andel)

      treningssett_markert, _, treningssett_umarkert, treningssett\
      = trening_testsett_oppdeling_dag(rå_data,
                                       trening,
                                       test_andel=ubenyttet_andel,
                                       ekskluder_kalkun_kylling=True)

  else:
    treningssett_markert = trening_markert
    treningssett_umarkert = trening_umarkert
    treningssett = trening

  enzym_treningssett_markert = enzymfordeling(rå_data,
                                              treningssett_markert)

  print("Enzymtyper Fordeling: Treningssett markert andel:", trenings_andel)
  print(enzym_treningssett_markert)

  # Finner antall av enzymtype med minst antall
  enzymtype_min = enzym_treningssett_markert['Frekvens'].min()

  # Splitter etter minst antall hvis lavere enn 10. Ellers 10
  splitt = 10 if enzymtype_min >= 10 else enzymtype_min

  veiledet_modell_evaluering_2(test_dict_res=dict_res_2_design_test,
                                trening_dict_res=dict_res_2_design_trening,
                                rå_data=rå_data,
                                treningssett_markert=treningssett_markert,
                                treningssett_umarkert=treningssett_umarkert,
                                treningssett=treningssett,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Enzymtyper Fordeling: Test
    Frekvens  Andel
E         15  22.73
B         13  19.70
C         13  19.70
D         12  18.18
A2         9  13.64
A1         4   6.06


Lagring av resultater

In [ ]:
mw_resultater_2_design_trening = pd.DataFrame(dict_res_2_design_trening)
mw_resultater_2_design_test = pd.DataFrame(dict_res_2_design_test)

mw_resultater_2_design_trening.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_trening.csv')
mw_resultater_2_design_test.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_test.csv')

# NIR målinger ekskludert

In [19]:
# Ekskluderer NIR målinger fra datasettene
NIR_kolonner = ['NIRfat', 'NIRwater', 'NIRash']
data_uten_NIR = data.drop(columns=NIR_kolonner)
data_design_uten_NIR = data_design.drop(columns=NIR_kolonner)

## Hele datasett

## Alternativ 1: Hele treningssett

In [20]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, \
trening_umarkert, enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_uten_NIR,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

Enzymtyper Fordeling: Test
            Frekvens  Andel
EnzymeCode                 
A2                31  34.07
E                 15  16.48
B                 14  15.38
C                 14  15.38
D                 13  14.29
A1                 4   4.40


In [21]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_hel_test_NIR,
                             trening_dict_res=dict_res_1_hel_trening_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

[I 2024-04-15 20:36:02,297] A new study created in memory with name: no-name-7064d42e-eba7-4075-b3b3-ea47318749e8
[I 2024-04-15 20:36:06,545] Trial 0 finished with value: 869.9265831940542 and parameters: {'n_estimators': 493, 'criterion': 'friedman_mse', 'max_depth': 15, 'min_samples_leaf': 3, 'max_features': 0.5170424905316815}. Best is trial 0 with value: 869.9265831940542.
[I 2024-04-15 20:36:09,431] Trial 2 finished with value: 1226.662628802649 and parameters: {'n_estimators': 595, 'criterion': 'squared_error', 'max_depth': 2, 'min_samples_leaf': 3, 'max_features': 0.5182616010122896}. Best is trial 0 with value: 869.9265831940542.
[I 2024-04-15 20:36:10,350] Trial 1 finished with value: 940.115937778626 and parameters: {'n_estimators': 984, 'criterion': 'squared_error', 'max_depth': 4, 'min_samples_leaf': 2, 'max_features': 0.6452999547188153}. Best is trial 0 with value: 869.9265831940542.
[I 2024-04-15 20:36:12,601] Trial 3 finished with value: 922.2407584022706 and parameters

Velger beste modell

## Alternativ 2: Hele treningssett

In [ ]:
# Definerer trenings- ogtestandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Parametervalg for kryssvalidering
splitt = 10

# Antall runder til optimalisering
n_runder = 100

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_hel_test_NIR,
                            trening_dict_res=dict_res_2_hel_trening_NIR,
                            rå_data=rå_data,
                            treningssett_markert=trening_markert,
                            treningssett_umarkert=trening_umarkert,
                            treningssett=trening,
                            testsett=testsett,
                            splitt=splitt,
                            n_runder=n_runder,
                            test_andel=test_andel,
                            trenings_andel=trenings_andel,
                            random_seed=random_seed)

Velger beste modell

## Designproduksjon

## Alternativ 1: Hele treningssettet

In [ ]:
# Setter fast testandel og treningsandel
test_andel = 0.2
trenings_andel = 1.0


# Deler datasettet i trening og testsett
trening_markert, enzymtyper_markert, testsett, enzymtyper_test, trening_umarkert,\
enzymtyper_umarkert, trening\
= trening_testsett_oppdeling_enzym(rå_data,
                                   data_design_uten_NIR,
                                   test_andel=test_andel)

#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10
gjentagelser = 10


# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_1(test_dict_res=dict_res_1_design_test_NIR,
                             trening_dict_res=dict_res_1_design_trening_NIR,
                             treningssett_markert=trening_markert,
                             enzymtyper_treningssett_markert=enzymtyper_markert,
                             treningssett_umarkert=trening_umarkert,
                             enzymtyper_treningssett_umarkert=enzymtyper_umarkert,
                             treningssett=trening,
                             testsett=testsett,
                             splitt=splitt,
                             gjentagelser=gjentagelser,
                             n_runder=n_runder,
                             test_andel=test_andel,
                             trenings_andel=trenings_andel,
                             random_seed=random_seed)

Velger beste modell

## Alternativ 2: Hele treningssettet

In [ ]:
# Definerer trenings- ogtestandel
trenings_andel = 1.0
test_andel = 0.2

# Deler datasettet i trening og testsett
trening_markert, testsett, trening_umarkert, trening\
= trening_testsett_oppdeling_dag(rå_data,
                                 data_design_uten_NIR,
                                 test_andel=test_andel,
                                 ekskluder_kalkun_kylling=True)


#Oversikt over fordelingen av enzymtyper i testsett
print("Enzymtyper Fordeling: Test")
print(enzymfordeling(rå_data, testsett))

In [ ]:
# Antall runder til optimalisering
n_runder = 100

# Parametervalg for kryssvalidering
splitt = 10

# Modelloptimalisering, trening og evaluering av RFR, KNR og SVR
veiledet_modell_evaluering_2(test_dict_res=dict_res_2_design_test_NIR,
                                trening_dict_res=dict_res_2_design_trening_NIR,
                                rå_data=rå_data,
                                treningssett_markert=trening_markert,
                                treningssett_umarkert=trening_umarkert,
                                treningssett=trening,
                                testsett=testsett,
                                splitt=splitt,
                                n_runder=n_runder,
                                test_andel=test_andel,
                                trenings_andel=trenings_andel,
                                random_seed=random_seed)

Velger beste modell

## Resultater

### Trening og valideringsresultater

In [ ]:
mw_resultater_1_hel_trening_NIR = pd.DataFrame(dict_res_1_hel_trening_NIR)
mw_resultater_2_hel_trening_NIR  = pd.DataFrame(dict_res_2_hel_trening_NIR)
mw_resultater_1_design_trening_NIR   = pd.DataFrame(dict_res_1_design_trening_NIR)
mw_resultater_2_design_trening_NIR   = pd.DataFrame(dict_res_1_design_trening_NIR)

### Testresultater

In [ ]:
mw_resultater_1_hel_test_NIR = pd.DataFrame(dict_res_1_hel_test_NIR)
mw_resultater_2_hel_test_NIR  = pd.DataFrame(dict_res_2_hel_test_NIR)
mw_resultater_1_design_test_NIR  = pd.DataFrame(dict_res_1_design_test_NIR)
mw_resultater_2_design_test_NIR = pd.DataFrame(dict_res_1_design_test_NIR)

### Lagring av resultater

In [ ]:
mw_resultater_1_hel_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_trening_NIR.csv')
mw_resultater_2_hel_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_trening_NIR.csv')

mw_resultater_1_design_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_trening_NIR.csv')
mw_resultater_2_design_trening_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_trening_NIR.csv')

mw_resultater_1_hel_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_hel_test_NIR.csv')
mw_resultater_2_hel_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_hel_test_NIR.csv')

mw_resultater_1_design_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_1_design_test_NIR.csv')
mw_resultater_2_design_test_NIR.to_csv('/content/drive/MyDrive/MasterV24/Bioco_data/mw_resultater_2_design_test_NIR.csv')